# Introduction to GPU Programming with Python
## Numba + CUDA: Advanced


Questions
* What is GPU shared memory ?
* How to use shared memory ? 

Objectives
* Understand where shared memory is located
* Understand how to create arrays of shared memory
* Learn to implement shared memory into kernels

### What is Shared memory ?
It's a memory located on GPU chip, not outside like a global memory. This is why it's extremely fast.

* Shared memory latency is 100x lower than that of global memory
* Allocated per thread block, so all threads in the block have access to the same shared memory
* Threads can access data in shared memory loaded from global memory by other threads within the same thread block


Another reason why threads are organized into blocks is to be able to introduce a shared memory concept.

A limited amount of shared memory can be allocated on the device to speed up access to data. That memory is shared amongst all threads in a given block. It's so much faster than the regular device memory. It also allows threads to cooperate on a given solution.

You can think of it as a manually-managed data cache.

Shared memory is also configurable. Shared memory resides in on-chip memory and shares space with L1 cache (registers).

### How to allocate shared memory in Numba

In [ ]:
 numba.cuda.shared.array(shape, type)

This function is called on the device, i.e. from the kernel or device function. A common pattern is to have each thread populate one element in the shared array, then wait for all threads to finish using syncthtreads:

### Thread synchronization
When sharing data between threads, we may need to avoid race condition: e.g. thread A is supposed to read data that is supposed to be written by thread B, but thread B have not finished writing that data and thread A is already trying to access it. 

To ensure correct results when parallel threads cooperate, we must synchronize the threads. CUDA provides a simple barrier synchronization primitive, __syncthreads(). A thread’s execution can only proceed past a __syncthreads() after all threads in its block have executed the __syncthreads(). 

### How to synchronize threads in Numba

In [ ]:
 numba.cuda.syncthreads()

### Main example: Matrix multiplication with shared memory

![](images/05-matmulshared.png)

### What's the Idea ? 
We use shared memory to re-use global memory data.

We decompose our algorithms into 2 phases:

    1. Reading data from global memory into tiles
    2. Looping over the elements in a tile and performing a dot product 
    
We also need to allocate memory for tiles (TBP,TBP) in shared memory 

In [1]:
import numpy as np
from numba import cuda

In [ ]:
#Part 3: Create a CUDA kernel with @cuda.jit decorator

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    
    # Define global and thread indices
    
    # Define number of blocks per grid
    
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        #####
        
        # Wait until all threads finish preloading
        
        # Computes partial product on the shared memory
        for j in range(TPB):
            #####
            
        # Wait until all threads finish computing
        
    # Put tmp into C matrix

In [ ]:
#Part 1: Create matrices A,B,C as numpy arrays (size 128x128). Fill A and B with random numbers.

In [ ]:
#Part 2: Calculate number of blocks and threads

In [ ]:
#Part 4: Call the kernel function and time it to get the execution time

### Exercise: Array reversal with shared memory

Here we re-use the code from [previous notebook](04-numba-cuda.ipynb) and add shared memory into play

In [2]:
from numba import cuda

In [ ]:
# Take this code and re-write it in the next cell by using a shared memory 
@cuda.jit
def reverseArrayBlock(d_out,d_in):
    ind_in = cuda.blockDim.x*cuda.blockIdx.x + cuda.threadIdx.x; ## Index of the current thread
    ind_out = cuda.gridsize(1)-ind_in-1 ## Total number of threads - in -1
    if ind_in<d_in.size:
        d_out[ind_out] = d_in[ind_in]

In [ ]:
# Part 2: Here is the code with shared memory
@cuda.jit
def reverseArrayBlock_shared(d_out,d_in):
    # Declare/allocate array s in shared memory
    ....
    # Create input index
    ....
    # Populate array s from arrat d_in
    ....
    # Synchronize threads in each block
    ....
    # Create output index
    ....
    if ind_in<d_in.size:
        # Populate output array d_out from shared array s
        ....

In [ ]:
dim=256*1000
NumThreads=128
NumBlocks = (dim + (NumThreads - 1)) // NumThreads

In [ ]:
#Part 1: Create arrays on CPU and GPU (if you want to)
a = np.arange(0,dim,dtype=np.int32)
b = np.zeros(dim,dtype=np.int32)
print(memSize)

In [ ]:
#Part 3: Call the kernel
# Static shared memory declaration
reverseArrayBlock_shared[NumBlocks,NumThreads](b,a)

In [ ]:
#Part 4: Modify the kernel as well as the call from the host by changing 
#        static shared memory declaration to dynamic
# Dynamic shared memory declaration
reverseArrayBlock_shared[NumBlocks,NumThreads,0,memSize](b,a)

## Key points
* **Numba CUDA Shared memory** 
    * Device (GPU) won't work without a Host(CPU)
    * Both Host and Device have their own memory
* **Kernel and Device functions**
    * Kernel is declared with @cuda.jit. Kernel is called from  the Host
    * Device function is declared with @cuda.jit(device=True) and is called from the Device.
* **Explicit data transfers between CPU and GPU**
    * 